# Brouillon

__Ce notebook est un brouillon de notre travail. Il ne fait donc pas partie de notre rapport.__

In [1]:
import os
import csv
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
import sklearn 

from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve
from sklearn.utils import shuffle
from math import *

from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

from PyPDF2 import PdfFileReader, PdfFileMerger
from pandas_datareader._utils import RemoteDataError

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from boruta import BorutaPy

ModuleNotFoundError: No module named 'pandas_datareader'

In [2]:
if not os.path.exists("Contenus") : 
    os.mkdir("Contenus")
    
if not os.path.exists("Contenus\Donnees") : 
    os.mkdir("Contenus\Donnees")
    
if not os.path.exists("Contenus\Donnees\Donnees_Traitees") : 
    os.mkdir("Contenus\Donnees\Donnees_Traitees")
    
if not os.path.exists("Contenus\Donnees\Donnees_brutes") : 
    os.mkdir("Contenus\Donnees\Donnees_brutes")
    
# Création du dossier de cac40
if not os.path.exists("Contenus\Graphiques") : 
    os.mkdir("Contenus\Graphiques")
    

folder_data_raw= r'Contenus\Donnees\Donnees_brutes'
folder_data_clean=r'Contenus\Donnees\Donnees_Traitees'
folder_graphique=r'Contenus\Graphiques'
if not os.path.exists(folder_data_raw) : os.mkdir(folder_data_raw)

Algorithmes_consideres=['RF','SVM','Logit']

In [3]:
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r
    print(r)
list_files(folder_data_raw)

[]

# Travail sur la base concatenée avec indicatrice de couleur du vin

In [4]:
winequality_red=pd.DataFrame.from_csv(list_files(folder_data_raw)[0],index_col=False,header=0, sep=';', parse_dates=False, encoding=None, tupleize_cols=False, infer_datetime_format=True)
winequality_white=pd.DataFrame.from_csv(list_files(folder_data_raw)[1],index_col=False,header=0, sep=';', parse_dates=False, encoding=None, tupleize_cols=False, infer_datetime_format=True)
winequality_red["red"]=1
winequality_white["red"]=0
winequality = pd.concat([winequality_white,winequality_red])
Variables=[X for X in winequality]
#Création de toutes les interactions possibles
Variables.remove("quality")
for X in Variables:
    for Y in Variables :
        winequality[X+"*"+Y]=winequality[X]*winequality[Y]
#Normalisation de toutes les interactions possibles
Variables_Finales=[X for X in winequality]
Variables_Finales.remove("quality")
#X_train, X_test, Y_train, Y_test = train_test_split(winequality.loc[:, winequality.columns != 'quality'],winequality["quality"])
Liste_Variables=[X for X in Variables_Finales]

NameError: name 'pd' is not defined

In [5]:
def Dico(Algo) :
    global Accuracy
    Accuracy={}
    Accuracy.clear()
    for Nom_Algotithme in Algo :
            Accuracy[Nom_Algotithme]={}
            Accuracy[Nom_Algotithme]['Selection_Variable']={}
            Accuracy[Nom_Algotithme]['Selection_Variable']['Oui']=[]
            Accuracy[Nom_Algotithme]['Selection_Variable']['Non']=[]

In [6]:
Dico(Algorithmes_consideres)


In [7]:
def Selection_Train_Test(Base,Nom_y,Boruta,Algorithme_Boruta, Algotithme_Entraine,Nom_Algotithme):
    #Base : Base à splitter
    #Nom_y : Nom de la variable d'intérêt
    #Boruta : Activer la sélection de variables avec Boruta ? 'Oui' ou 'Non'
    #Algorithme_Boruta : Mettre l'algorithme sur lequel repose le choix de Boruta (mettre le classi Sklearn)
    #Algotithme_Entraine : Choisir l'algorithme entrainé et servant à tester 
    #Nom_Algotithme : Abréviation du nom de l'algo utilisé. Doit obligatoirement être défini dans la liste 'Algorithmes_consideres'
                      #définie dans la cellule 2 du notebook
    if Nom_Algotithme not in Algorithmes_consideres :
        print("Le nom de l'algorithme doit être dans la liste 'Algo'")
    if Boruta not in ["Oui","Non"]:
        print("L'entrée Boruta doit prendre la valeur 'Oui' ou 'Non'")
    X_train, X_test, Y_train, Y_test = train_test_split(Base.loc[:, Base.columns != Nom_y],Base[Nom_y])
    x_train = X_train.values
    y_train = Y_train.values
    y_train = y_train.ravel()
    y_test = Y_test.values
    y_test = y_test.ravel()
    
    if Boruta=='Oui' :
        # define Boruta feature selection method
        feat_selector = BorutaPy(Algorithme_Boruta, n_estimators=100, verbose=2, random_state=1)

        # find all relevant features - 5 features should be selected
        feat_selector.fit(x_train, y_train)

        # check selected features
        feat_selector.support_

        # check ranking of features
        feat_selector.ranking_

        selected_train = x_train[:,feat_selector.support_]
        print ("")
        print ("Selected Feature Matrix Shape")
        print (selected_train.shape)

        Select=[X for X in feat_selector.support_]
        Liste_Var_Selectionnees=[]
        for x in range(0,len(Select)) :
            Var=Select[x]
            if Var==True :
                Liste_Var_Selectionnees.append(Liste_Variables[x])
        print(Liste_Var_Selectionnees)
        selected_train = pd.DataFrame(selected_train)

        for x in range(0,len(selected_train.columns)) :
                selected_train = selected_train.rename(columns={x: Liste_Var_Selectionnees[x]})
        selected_test=X_test
        for X in X_test.columns:
            if X not in selected_train.columns :
                selected_test=selected_test.drop(X,1)

        clf=Algotithme_Entraine
        clf.fit(selected_train, y_train)
        Accuracy[Nom_Algotithme]['Selection_Variable'][Boruta].append(clf.score(selected_test,y_test))
        
    if Boruta=="Non" :
        x_test = X_test.values
        clf=Algotithme_Entraine
        clf.fit(x_train, y_train)
        Accuracy[Nom_Algotithme]['Selection_Variable'][Boruta].append(clf.score(x_test,y_test))

In [8]:
Selection_Train_Test(winequality,'quality','Oui',RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,
max_features='auto', max_leaf_nodes=None, bootstrap=True,oob_score=False, random_state=None, verbose=0, n_jobs=-1),RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,max_features='auto', max_leaf_nodes=None, bootstrap=True,oob_score=False, random_state=None, verbose=0, n_jobs=-1),'RF')

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	156
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	27
Tentative: 	44
Rejected: 	85


C:\Users\olivi\Desktop\Logiciel Pro\PythonEnsae\python\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning:

invalid value encountered in greater



Iteration: 	9 / 100
Confirmed: 	27
Tentative: 	44
Rejected: 	85


C:\Users\olivi\Desktop\Logiciel Pro\PythonEnsae\python\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning:

invalid value encountered in greater



Iteration: 	10 / 100
Confirmed: 	27
Tentative: 	44
Rejected: 	85


...

Iteration: 	97 / 100
Confirmed: 	47
Tentative: 	9
Rejected: 	100


C:\Users\olivi\Desktop\Logiciel Pro\PythonEnsae\python\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning:

invalid value encountered in greater



Iteration: 	98 / 100
Confirmed: 	47
Tentative: 	9
Rejected: 	100


C:\Users\olivi\Desktop\Logiciel Pro\PythonEnsae\python\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning:

invalid value encountered in greater



Iteration: 	99 / 100
Confirmed: 	47
Tentative: 	9
Rejected: 	100


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	47
Tentative: 	4
Rejected: 	100

Selected Feature Matrix Shape
(4872, 47)
['volatile acidity', 'density', 'alcohol', 'fixed acidity*volatile acidity', 'fixed acidity*chlorides', 'fixed acidity*total sulfur dioxide', 'volatile acidity*fixed acidity', 'volatile acidity*chlorides', 'volatile acidity*total sulfur dioxide', 'volatile acidity*density', 'volatile acidity*pH', 'volatile acidity*alcohol', 'citric acid*alcohol', 'residual sugar*free sulfur dioxide', 'residual sugar*sulphates', 'chlorides*fixed acidity', 'chlorides*volatile acidity', 'chlorides*residual sugar', 'chlorides*total sulfur dioxide', 'chlorides*density', 'free sulfur dioxide*residual sugar', 'free sulfur dioxide*sulphates', 'free sulfur dioxide*alcohol', 'total sulfur dioxide*volatile acidity', 'total sulfur dioxide*chlorides', 'density*volatile acidity', 'density*chlorides', 'density*density

In [9]:
Selection_Train_Test(winequality,'quality','Non',RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,
max_features='auto', max_leaf_nodes=None, bootstrap=True,oob_score=False, random_state=None, verbose=0, n_jobs=-1),RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,max_features='auto', max_leaf_nodes=None, bootstrap=True,oob_score=False, random_state=None, verbose=0, n_jobs=-1),'RF')

In [10]:
Accuracy

{'Logit': {'Selection_Variable': {'Non': [], 'Oui': []}},
 'RF': {'Selection_Variable': {'Non': [0.67692307692307696],
   'Oui': [0.68184615384615388]}},
 'SVM': {'Selection_Variable': {'Non': [], 'Oui': []}}}

In [ ]:
# Travail sur les deux bases de couleurs différentes